In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [4]:

from utils import dasker, helper, config
from ml.models.base.slug_ann import SlugANN
from dask_ml.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score


from sklearn import preprocessing
from ml.models import common
from dask_ml.preprocessing import StandardScaler

In [5]:
client = dasker.get_local_client()

2022-11-02 11:56:53,122 - asyncio - DEBUG - Using selector: EpollSelector


2022-11-02 11:56:54,817 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-vygkgzvq', purging
2022-11-02 11:56:54,817 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-6zrrv9gm', purging
2022-11-02 11:56:54,817 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-iaqg60b7', purging
2022-11-02 11:56:54,818 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-uo1ymmfi', purging


In [6]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)
X_scalar = StandardScaler().fit_transform(X.copy())


X_train_scalar, X_test_scalar, y_train, y_test = train_test_split(X_scalar, 
                                                                                                                                y,
                                                                                                                                test_size=0.33,
                                                                                                                                random_state=config.rand_state)



In [7]:
config.project_name = '/covid_slug_ann'

In [8]:
slug_ann = SlugANN('slug_ann', X_train_scalar, X_test_scalar, y_train_scalar, y_test_scalar)
slug_ann.epochs = 100
slug_ann.num_reruns = 2



In [10]:
slug_ann.fetch_model()

2022-11-02 12:10:41,854 - console_info - INFO - slug ann: Starting train for trials:100 with epochs: 100
2022-11-02 12:10:41,854 - console_info - INFO - slug ann: Starting train for trials:100 with epochs: 100
2022-11-02 12:10:41,859 - console_info - INFO - slug ann: Cleared previous models in the model save path
2022-11-02 12:10:41,859 - console_info - INFO - slug ann: Cleared previous models in the model save path
2022-11-02 12:18:53,117 - console_info - INFO - slug xgboost: Number of trials: 100
2022-11-02 12:18:53,117 - console_info - INFO - slug xgboost: Number of trials: 100
2022-11-02 12:18:53,151 - console_info - INFO - Best trial: 83
2022-11-02 12:18:53,151 - console_info - INFO - Best trial: 83
2022-11-02 12:18:53,218 - console_info - INFO -   test r2 score: 0.4396390656534799
2022-11-02 12:18:53,218 - console_info - INFO -   test r2 score: 0.4396390656534799
2022-11-02 12:18:53,227 - console_info - INFO - slug_ann: Model saved at /mnt/c/Users/rwmas/GitHub/xai/xai_api/app/ml/

Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.32452014999170276, inplace=False)
  (3): Linear(in_features=50, out_features=181, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.32452014999170276, inplace=False)
  (6): Linear(in_features=181, out_features=234, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.32452014999170276, inplace=False)
  (9): Linear(in_features=234, out_features=176, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.32452014999170276, inplace=False)
  (12): Linear(in_features=176, out_features=57, bias=True)
  (13): ReLU()
  (14): Dropout(p=0.32452014999170276, inplace=False)
  (15): Linear(in_features=57, out_features=172, bias=True)
  (16): ReLU()
  (17): Dropout(p=0.32452014999170276, inplace=False)
  (18): Linear(in_features=172, out_features=177, bias=True)
  (19): ReLU()
  (20): Dropout(p=0.32452014999170276, inplace=False)
  (21): Linear(in_features=177, out_features=1, bias=True)
  (22): ReLU()
)

In [17]:
slug_ann.score

[0.6823503468291505, 0.4396390656534799]

In [19]:
pred = slug_ann.predict(X)
r2_score(pred, y)


-0.07226272443628745

In [13]:
threshold=0.1
slug_ann.load_score(threshold=threshold)
# slug_ann.load_score()

[0.6823503468291505, 0.4396390656534799]

In [14]:
slug_ann.score

[0.6823503468291505, 0.4396390656534799]

In [15]:
pred = slug_ann.predict(X_scalar)
r2_score(pred, y)

-0.07226272443628745

In [ ]:
# brisk_xgb.empty_media()
common.save_state(slug_ann, slug_ann.model_file_name + '.pickle')

In [ ]:
slug_ann_reinstate = common.load_state('slug_ann.pickle')

In [ ]:
slug_ann_reinstate.load_score()

In [ ]:
pred = slug_ann_reinstate.predict(X)
r2_score(pred, y)